In [2]:
# create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Actual_To_SR_Actual") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/25 09:03:07 WARN Utils: Your hostname, codespaces-780a6a, resolves to a loopback address: 127.0.0.1; using 10.0.2.244 instead (on interface eth0)
25/07/25 09:03:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/25 09:03:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
actual = spark.read.csv("/workspaces/my_development_codes/notebooks/data_actual_pandorasractual/Actual.csv", header=True, inferSchema=True)
actual.show(20)


+----------------------+-----------+--------------------+--------------------------+---------------+------------------------+-------------------+---------+-----------------------------+-----------+-----------------+------+
|Version.[Version Name]|Item.[Item]|   Channel.[Channel]|Channel.[Planning Channel]|Region.[Region]|Region.[Planning Region]|Location.[Location]|PnL.[PnL]|Demand Domain.[Demand Domain]| Time.[Day]|Account.[Account]|Actual|
+----------------------+-----------+--------------------+--------------------------+---------------+------------------------+-------------------+---------+-----------------------------+-----------+-----------------+------+
|    CurrentWorkingView|  792352C01|Non O&O Concept S...|                   Non O&O|     RS10037680|                      RS|                All|      All|                          All|17-Jul-2023|              All|     2|
|    CurrentWorkingView|  792352C01|Non O&O Concept S...|                   Non O&O|     RS10037680|        

In [18]:
ecom = spark.read.csv("/workspaces/my_development_codes/notebooks/data_actual_pandorasractual/ecom_assoc.csv", header=True, inferSchema=True)
ecom.show(5)

+----------------------+--------------------------+---------------+----------------------+
|Version.[Version Name]|Channel.[Planning Channel]|Region.[Region]|Store Ecom Association|
+----------------------+--------------------------+---------------+----------------------+
|    CurrentWorkingView|                O&O Online|     SK10060522|            SK10060522|
|    CurrentWorkingView|                O&O Online|     RS10037680|                BEECOM|
|    CurrentWorkingView|                O&O Online|     TR10058894|            TR10058894|
|    CurrentWorkingView|                O&O Online|     CZ10058744|            CZ10058744|
|    CurrentWorkingView|                O&O Online|     PL10036905|                BEECOM|
+----------------------+--------------------------+---------------+----------------------+
only showing top 5 rows


In [19]:
customer_group = spark.read.csv("/workspaces/my_development_codes/notebooks/data_actual_pandorasractual/region_statchannel.csv", header=True, inferSchema=True)
customer_group.show(5)

+------------------------+--------------------+----------------------+
|Region.[Planning Region]|Region.[DisplayName]|Channel.[Stat Channel]|
+------------------------+--------------------+----------------------+
|                      OM|                Oman|               Non O&O|
|                      OM|                Oman|          O&O Physical|
|                      OM|                Oman|            O&O Online|
|                      PL|              Poland|               Non O&O|
|                      PL|              Poland|          O&O Physical|
+------------------------+--------------------+----------------------+
only showing top 5 rows


In [20]:
from pyspark.sql.functions import concat, col,lit
from pyspark.sql import functions as F
customer_group.withColumn("[Sales Domain].[Customer Group]",concat(col("`Region.[DisplayName]`"),lit("_"),col("`Channel.[Stat Channel]`"))).show(5)

# fcst1 = fcst1.merge(ecom[["Region.[Region]","Channel.[Planning Channel]", "Store Ecom Association"]],how="left",on=["Region.[Region]","Channel.[Planning Channel]"])
# item*version*time*Activity1*location
# O&O physical store, Region.Region->Activity1.To Location
# O&O Online store, Region.Region->Store Ecom Ass mapped    
# Select ([Version].[Version Name] * [Item].[Item] * [Location].[Location] * [Activity1].[Activity1] * [Time].[Week] ) on row, ({Measure.[PandoraSR W Actuals]}) on column;

+------------------------+--------------------+----------------------+-------------------------------+
|Region.[Planning Region]|Region.[DisplayName]|Channel.[Stat Channel]|[Sales Domain].[Customer Group]|
+------------------------+--------------------+----------------------+-------------------------------+
|                      OM|                Oman|               Non O&O|                   Oman_Non O&O|
|                      OM|                Oman|          O&O Physical|              Oman_O&O Physical|
|                      OM|                Oman|            O&O Online|                Oman_O&O Online|
|                      PL|              Poland|               Non O&O|                 Poland_Non O&O|
|                      PL|              Poland|          O&O Physical|            Poland_O&O Physical|
+------------------------+--------------------+----------------------+-------------------------------+
only showing top 5 rows


In [ ]:
# there are 2 dataframes , ecom and actual , you have to bring the Store Ecom Association from the ecom and put into actual

# Step 1: Filter for O&O Online and select relevant columns from ecom
ecom_filtered = ecom.filter(F.col("`Channel.[Planning Channel]`") == "O&O Online") \
                   .select(F.col("`Region.[Region]`").alias("region_key"), "Store Ecom Association")


# step 2 : left join with actual on region.region with ecom wherever a match

# drop "Non O&O" values from Channel.Planning Channel
actual = actual.filter(actual["`Channel.[Planning Channel]`"] != "Non O&O")

joined_df = actual.alias("a").join(
    ecom_filtered.alias("e"),
    on=F.col("a.`Region.[Region]`") == F.col("e.region_key"), # Join condition using aliases
    how="left"
).withColumn("Region.[Region]",
    F.when(F.col("e.Store Ecom Association").isNotNull(), F.col("e.Store Ecom Association"))
    .otherwise(F.col("a.`Region.[Region]`"))
).drop(F.col("e.region_key"), "Store Ecom Association")


joined_df.show(10)



+----------------------+-----------+--------------------+--------------------------+---------------+------------------------+-------------------+---------+-----------------------------+-----------+-----------------+------+
|Version.[Version Name]|Item.[Item]|   Channel.[Channel]|Channel.[Planning Channel]|Region.[Region]|Region.[Planning Region]|Location.[Location]|PnL.[PnL]|Demand Domain.[Demand Domain]| Time.[Day]|Account.[Account]|Actual|
+----------------------+-----------+--------------------+--------------------------+---------------+------------------------+-------------------+---------+-----------------------------+-----------+-----------------+------+
|    CurrentWorkingView|  792352C01|Non O&O Concept S...|                   Non O&O|         BEECOM|                      RS|                All|      All|                          All|17-Jul-2023|              All|     2|
|    CurrentWorkingView|  792352C01|Non O&O Concept S...|                   Non O&O|         BEECOM|        